## Transfer Learning
What is it? We will take exist model which has learned from another problem and using them for our own problem.

In [1]:
# Are we using a GPU?
!nvidia-smi

Sat May 14 22:33:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.13       Driver Version: 462.13       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 307... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8    14W /  N/A |    151MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# TensorFlow Hub
There are tons of existing model created by people, researchers.
https://tfhub.dev/

We can get better results with only 10% of the data...



In [2]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Get data (10% of labels)
import zipfile



# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [4]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

# Create data loaders (preparing the data)


In [5]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = '10_food_classes_10_percent/test/'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
target_size=IMAGE_SHAPE,
batch_size=BATCH_SIZE,
class_mode='categorical')

print("Testing images:")
test_data= train_datagen.flow_from_directory(test_dir,
target_size=IMAGE_SHAPE,
batch_size=BATCH_SIZE,
class_mode='categorical')


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


# Setting up callbacks (things to run whilst our model trains)

Callbacks -> extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks include:

<b>Experiment tracking with TensorBoard</b> - log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters). Helpful to compare the results of different models on your data.\
<b>Model checkpointing</b> - save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting.\
<b>Early stopping</b> - leave your model training for an arbitrary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take.

We'll see how the TensorBoard callback can be used.

The TensorBoard callback can be accessed using tf.keras.callbacks.TensorBoard()\

We will track our modelling experiments using TensorBoard.

In [6]:
# Create tensorboard callback
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

Because you're likely to run multiple experiments. It's a good idea to be able to track them in some way!

# Create model using TensorFlow Hub

We will be using two models from TensorFlow Hub:\
ResNetV2\
EfficientNet

# Types of Transfer Learning

<b>Transfer Learning </b> -> Take a pretrained model as it it and apply it to your task without any changes\

<b>Feature extraction transfer learning </b> -> Take the weight of pretrained model has learned and adjust its outputs to be more suited to your problem.\

<b>Fine-tuning transfer Learning</b> Take weight of pretrained model and adjust them to your own problem.\

A common workflow is to <b>freeze</b> all of the learned patterns in the bottom layers of a pretrained model so they're untrainable, then you can train and make adjustment of the outputs to your own data.

### Lets get started!

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"



# EfficientNetB0 feature vector (version 2)
efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [9]:
def create_model(model_url, num_classes=10):
    """Take a TensorFlow Hub URL and create Keras Sequential model with it.

    Returns:
    uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs
    """
    # Download pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
    trainable=False, # freeze the underlying patterns
    name='feature_extraction_layer',
    input_shape=IMAGE_SHAPE+(3,)) # define the input image shape

    # Create our own model
    model = tf.keras.Sequential([
        feature_extractor_layer, # use feature extraction layer as the base
        layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer
    ])

    return model


In [10]:
# Create model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# Compile
resnet_model.compile(loss='categorical_crossentropy',
optimizer=tf.keras.optimizers.Adam(),
metrics=['accuracy'])

In [11]:
# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  # Add TensorBoard callback to model (callbacks parameter takes a list)
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", # save experiment logs here
                                                                         experiment_name="resnet50V2")]) # name of log files

Saving TensorBoard log files to: tensorflow_hub/resnet50V2/20220514-223350
Epoch 1/5
24/24 [==============================] - 31s 889ms/step - loss: 1.9470 - accuracy: 0.3373 - val_loss: 1.1802 - val_accuracy: 0.6380
Epoch 2/5
24/24 [==============================] - 6s 276ms/step - loss: 0.9368 - accuracy: 0.7280 - val_loss: 0.8522 - val_accuracy: 0.7196
Epoch 3/5
24/24 [==============================] - 6s 263ms/step - loss: 0.6501 - accuracy: 0.8120 - val_loss: 0.7634 - val_accuracy: 0.7528
Epoch 4/5
24/24 [==============================] - 6s 253ms/step - loss: 0.5010 - accuracy: 0.8600 - val_loss: 0.7056 - val_accuracy: 0.7692
Epoch 5/5
24/24 [==============================] - 6s 254ms/step - loss: 0.3954 - accuracy: 0.9040 - val_loss: 0.6698 - val_accuracy: 0.7796


we achieve 78% with only 5 epochs and 10% of training data...

In [12]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 2048)              23564800  
_________________________________________________________________
output_layer (Dense)         (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


This means during training the model updates the 20,490 parameters in the ouput layer to suit our dataset

In [13]:
# Create model
efficientnet_model = create_model(model_url=efficientnet_url, # use EfficientNetB0 TensorFlow Hub URL
                                  num_classes=train_data_10_percent.num_classes)

# Compile EfficientNet model
efficientnet_model.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

# Fit EfficientNet model 
efficientnet_history = efficientnet_model.fit(train_data_10_percent, # only use 10% of training data
                                              epochs=10, # train for 10 epochs
                                              steps_per_epoch=len(train_data_10_percent),
                                              validation_data=test_data,
                                              validation_steps=len(test_data),
                                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                                     # Track logs under different experiment name
                                                                                     experiment_name="efficien")])

Saving TensorBoard log files to: tensorflow_hub/efficien/20220514-223501
Epoch 1/10
24/24 [==============================] - 29s 949ms/step - loss: 1.9577 - accuracy: 0.3653 - val_loss: 1.5268 - val_accuracy: 0.6132
Epoch 2/10
24/24 [==============================] - 7s 280ms/step - loss: 1.3072 - accuracy: 0.6960 - val_loss: 1.1335 - val_accuracy: 0.7032
Epoch 3/10
24/24 [==============================] - 6s 240ms/step - loss: 1.0056 - accuracy: 0.7587 - val_loss: 0.9549 - val_accuracy: 0.7420
Epoch 4/10
24/24 [==============================] - 6s 275ms/step - loss: 0.8397 - accuracy: 0.8080 - val_loss: 0.8619 - val_accuracy: 0.7548
Epoch 5/10
24/24 [==============================] - 6s 263ms/step - loss: 0.7336 - accuracy: 0.8293 - val_loss: 0.8043 - val_accuracy: 0.7640
Epoch 6/10
24/24 [==============================] - 6s 276ms/step - loss: 0.6564 - accuracy: 0.8467 - val_loss: 0.7627 - val_accuracy: 0.7724
Epoch 7/10
24/24 [==============================] - 7s 282ms/step - loss: 